# Imports

## Environment Variables

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
ANTHROPIC_MODEL = os.getenv("ANTHROPIC_MODEL")
OPENAI_MODEL = os.getenv("OPENAI_MODEL")

# Verify keys are loaded
print(f'Anthropic API Key loaded: {bool(os.getenv("ANTHROPIC_API_KEY"))}\nAnthropic model: {os.getenv("ANTHROPIC_MODEL")}')
print(f'OpenAI API Key loaded: {bool(os.getenv("OPENAI_API_KEY"))}\nOpenAI model: {os.getenv("OPENAI_MODEL")}')
print(OPENAI_MODEL)

Anthropic API Key loaded: True
Anthropic model: claude-sonnet-4-20250514
OpenAI API Key loaded: True
OpenAI model: gpt-4o-mini
gpt-4o-mini


## Packages

In [2]:
from typing import Literal, TypedDict, Annotated
from datetime import datetime
import json
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, AnyMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode

# Configure LLM
&rarr; choose anthropic / openai 

In [3]:
def get_llm(provider: Literal["anthropic", "openai"] = "openai"):
    """Get LLM instance based on provider choice"""
    if provider == "anthropic":
        base_llm = ChatAnthropic(
            model=ANTHROPIC_MODEL,
            temperature=0.7,
            max_tokens=1024
        )
    elif provider == "openai":
        base_llm = ChatOpenAI(
            model=OPENAI_MODEL,
            temperature=0.7,
            max_tokens=1024
        )
    else:
        raise ValueError(f"Unknown provider: {provider}")
    
    return base_llm


# Choose your provider here
LLM_PROVIDER = "openai"  # Change to "anthropic" if you prefer
llm = get_llm(LLM_PROVIDER)

print(f"Using {LLM_PROVIDER.upper()} as LLM provider")

Using OPENAI as LLM provider


# Tools

## Create tools

### get_current_datetime

In [4]:
# Define the datetime tool
@tool
def get_current_datetime() -> str:
    """Get the current date and time. Use this when you need to know today's date."""
    now = datetime.now()
    return f"Current date: {now.strftime('%Y-%m-%d')}, Current time: {now.strftime('%H:%M:%S')}"

## Give access to tools

### With bind_tools 
Possible but will be using the ToolBind option

In [5]:
# List of tools
tools = [get_current_datetime]
print(f"Tools available: {[t.name for t in tools]}")

Tools available: ['get_current_datetime']


In [6]:
# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

### With ToolNode

In [7]:
# Create ToolNode - handles tool execution automatically
tool_node = ToolNode(tools)

print(f"ToolNode created with tools: {[t.name for t in tools]}")

ToolNode created with tools: ['get_current_datetime']


# States 

In [8]:
# State = the data that flows through the workflow

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]  # LangGraph handles message merging
    name: str                       # Employee name
    start_date: str                 # Start date
    end_date: str                   # End date (optional)
    work_hours: float               # Weekly work hours
    salary: float                   # Salary
    info_complete: bool             # All info collected?
    human_decision: str             # "approve" or "reject"
    is_update: bool                 # New entry or update existing?

# Helper variables and functions

In [ ]:
# System prompt for the chatbot
SYSTEM_PROMPT = """You are Lola, a friendly HR assistant collecting employee information.

You have access to tools:
- get_current_datetime: Use this tool when you need to know today's date (e.g., if user says "today", "next monday", "in two weeks")

You need to collect these fields (one at a time, conversationally):
1. name - Employee's full name
2. start_date - Employment start date (normalize to YYYY-MM-DD)
3. end_date - Employment end date, can be empty if ongoing (normalize to YYYY-MM-DD or "ongoing")
4. work_hours - Weekly work hours (as a number)
5. salary - Annual salary in CHF (as a number)

Important rules:
- Be friendly and conversational
- Ask for ONE piece of information at a time
- When user mentions relative dates like "today", "tomorrow", "next week" - USE THE get_current_datetime TOOL first!
- The user may answer in ANY date format or language (e.g. "1st of April 2026", "1. April 2026", German or English).
- You must ALWAYS normalize dates to ISO format: YYYY-MM-DD.
- If no end date exists, output null.
- End date must always be after beginning date, without exception and no matter how sure the user say they are
- Employee names must be properly capitalized: only the first letter of each word should be uppercase,
  and the rest lowercase. Examples:
  * "Zacharias HAeusgen" → "Zacharias Haeusgen"
  * "john SMITH" → "John Smith"
  * "MARIA garcia" → "Maria Garcia"
- Salary can be provided in any format (e.g., "50k EUR a year", "40K USD a year", "30k CHF a year"). 
- Salary MUST ALWAYS be a positive number
- You must ALWAYS convert the salary to CHF (Swiss Francs) as an annual integer amount.
- Use these approximate conversion rates: 1 EUR = 0.95 CHF, 1 USD = 0.88 CHF, 1 GBP = 1.12 CHF
- work hours must always be positive. maximum work hours allowed is 50 hours. if the user insists it is more, mention that it is not legal. do not accept negative values or values larger than 50.
- Ask follow-up questions only if information is missing or unclear.

When you have ALL the information, respond with ONLY a JSON object:
{"name": "...", "start_date": "YYYY-MM-DD", "end_date": "YYYY-MM-DD or ongoing", "work_hours": number, "salary": number, "complete": true}

If information is still missing, just chat normally (no JSON).
"""

In [ ]:
# Keywords to exit the conversation
EXIT_KEYWORDS = {"exit", "bye", "quit", "stop", "cancel", "goodbye", "end"}

In [ ]:
# TODO keep this programmatic data check? or leave just the LLM ?
def validate_data(data: dict) -> list[str]:
    """Validate collected data. Returns list of error messages."""
    errors = []
    
    # 1. Minimum yearly salary 20000 CHF
    salary = data.get("salary", 0)
    if salary < 20000:
        errors.append(f"Salary must be at least 20,000 CHF (got {salary})")
    
    # 2. Start date must be after 2010
    start_date = data.get("start_date", "")
    if start_date:
        start_year = int(start_date.split("-")[0])
        if start_year <= 2010:
            errors.append(f"Start date must be after 2010 (got {start_date})")
    
    # 3. End date must be after start date (if not ongoing)
    end_date = data.get("end_date", "")
    if end_date and end_date.lower() != "ongoing" and start_date:
        if end_date <= start_date:
            errors.append(f"End date ({end_date}) must be after start date ({start_date})")
    
    return errors

In [ ]:
import re

def extract_json_from_text(text: str) -> dict | None:
    """Find and parse JSON object from text, even if surrounded by other content."""
    match = re.search(r'\{[^{}]*"complete"\s*:\s*true[^{}]*\}', text)
    if match:
        try:
            return json.loads(match.group())
        except json.JSONDecodeError:
            return None
    return None

# Nodes
NB: Chatbots are nodes

In [ ]:
# NODE 1: Chatbot with tool support

def chatbot(state: State) -> dict:
    """Chatbot node - calls LLM with tools, may request tools or collect data."""
    
    messages = state.get("messages", [])
    
    # --- EXIT CHECK (check last human message) ---
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            if msg.content.lower().strip() in EXIT_KEYWORDS:
                print("👋 Conversation ended by user.")
                return {"info_complete": True, "human_decision": "cancel"}
            break
    
    # --- BUILD LLM MESSAGES ---
    llm_messages = [SystemMessage(content=SYSTEM_PROMPT)] + messages
    
    # First message - greet user
    if not messages:
        llm_messages.append(HumanMessage(content="Hi, I need to enter employee information."))
    
    # --- CALL LLM WITH TOOLS ---
    response = llm_with_tools.invoke(llm_messages)
    
    # --- CHECK IF LLM WANTS TO USE TOOLS ---
    if response.tool_calls:
        print(f"🔧 LLM requesting tool: {response.tool_calls[0]['name']}")
        return {"messages": [response]}  # Workflow routes to ToolNode
    
    # --- NO TOOL CALL - PROCESS RESPONSE ---
    assistant_message = response.content
    print(f"🤖 Lola: {assistant_message}")
    
    # --- CHECK FOR COMPLETE JSON ---
    data = extract_json_from_text(assistant_message)
    
    if data and data.get("complete"):
        errors = validate_data(data)
        
        if errors:
            error_msg = "⚠️ Validation errors:\n" + "\n".join(f"  - {e}" for e in errors)
            print(error_msg)
            
            user_input = input("You: ")
            
            return {
                "messages": [response, AIMessage(content=error_msg), HumanMessage(content=user_input)],
                "info_complete": False,
            }
        
        print("✅ All information collected and validated!")
        return {
            "messages": [response],
            "name": data["name"],
            "start_date": data["start_date"],
            "end_date": data["end_date"],
            "work_hours": float(data["work_hours"]),
            "salary": float(data["salary"]),
            "info_complete": True,
        }
    
    # --- CONTINUE CONVERSATION ---
    user_input = input("You: ")
    
    return {
        "messages": [response, HumanMessage(content=user_input)],
        "info_complete": False,
    }

In [ ]:
# NODE 2: Human verification - asks for approval via input()
def human_verification(state: State) -> dict:
    """Shows collected data and asks human to approve or reject."""
    
    # all these variables can be accessed and be worked with
    print("\n" + "="*50)
    print("📋 PLEASE REVIEW THE DATA:")
    print("="*50)
    print(f"   Name:       {state.get('name')}")
    print(f"   Start date: {state.get('start_date')}")
    print(f"   End date:   {state.get('end_date')}")
    print(f"   Work hours: {state.get('work_hours')}")
    print(f"   Salary:     {state.get('salary')} CHF")
    print("="*50)
    
    decision = input("Type 'approve' or 'reject': ").lower().strip()
    
    if decision == "approve":
        print("✅ Approved!")
        return {"human_decision": "approve"}
    else:
        print("❌ Rejected - returning to chatbot for corrections")
        return {"human_decision": "reject"}

In [ ]:
# NODE 3: Create new entry
def create_entry(state: State) -> State:
    """Creates a new employee record"""
    
    # TODO: Add database insert logic here
    print(f"✅ [create_entry] Will create new entry for: {state.get('name')}")
    return state

In [ ]:
# NODE 4: Update existing entry  
def update_entry(state: State) -> State:
    """Updates an existing employee record"""
    
    # TODO: Add database update logic here
    print(f"✅ [update_entry] Will update entry for: {state.get('name')}")
    return state

# Routers

In [ ]:
# ROUTER 1: After chatbot - check if tools needed, then continue
def route_after_chatbot(state: State) -> str:
    """Routes based on: tool calls, cancellation, or info complete."""
    
    # Check for cancel
    if state.get("human_decision") == "cancel":
        return END
    
    # Check if last message has tool calls
    messages = state.get("messages", [])
    if messages:
        last_msg = messages[-1]
        if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
            return "tools"  # Route to ToolNode
    
    # Check if info complete
    if state.get("info_complete"):
        return "human_verification"
    
    return "chatbot"  # Continue collecting

In [ ]:
# ROUTER 2: After human verification - what did human decide?
def route_after_verification(state: State) -> str:
    decision = state.get("human_decision")
    
    if decision == "reject":
        return "chatbot"             # Rejected -> go back to fix
    
    if decision == "approve":
        if state.get("is_update"):
            return "update_entry"    # Update existing
        return "create_entry"        # Create new
    
    return "chatbot"                 # No decision -> go back

# Workflow

In [ ]:
# Build the graph (with ToolNode)
workflow = StateGraph(State)

# Add nodes
workflow.add_node("chatbot", chatbot)
workflow.add_node("tools", tool_node)                 # ToolNode handles tool execution
workflow.add_node("human_verification", human_verification)
workflow.add_node("create_entry", create_entry)
workflow.add_node("update_entry", update_entry)

# Add edges
workflow.add_edge(START, "chatbot")

workflow.add_conditional_edges(
    "chatbot",
    route_after_chatbot,
    ["chatbot", "tools", "human_verification", END]
)

workflow.add_edge("tools", "chatbot")                 # After tools, back to chatbot

workflow.add_conditional_edges(
    "human_verification", 
    route_after_verification,
    ["chatbot", "create_entry", "update_entry"]
)

workflow.add_edge("create_entry", END)
workflow.add_edge("update_entry", END)

print("Workflow created!")

In [ ]:
# Compile workflow
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

print("Workflow compiled!")

# Visualize graph

In [ ]:
# Visualize the workflow
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))

# Run the workflow

In [ ]:
# Run workflow (with tool support!)
initial_state = {
    "messages": [],
    "name": "",
    "start_date": "",
    "end_date": "",
    "work_hours": 0.0,
    "salary": 0.0,
    "info_complete": False,
    "human_decision": "",
    "is_update": False,
}

config = {"configurable": {"thread_id": "test-1"}}

# Run - try saying "start date is today" to test the tool!
for event in app.stream(initial_state, config):
    pass

print("\n🏁 Workflow complete!")

To Do       
 - add the other variables
 - ensure that limits are correct and that the user can't input whatever (consider keeping the programmatic validation of data, not only the llm)
 - add tool for vurrency conversion with current values
 - add langsmith to see consumption